# The Perceptron
Let us implement a perceptron.
As a reminder, our perceptron takes multiple inputs, weights each of them with a certain factor and checks if the sum is bigger than a threshold.
![Perceptron](./images/perceptron.png)
First we are going to collect the data to run this example:

In [ ]:
! wget -c --retry-connrefused --tries=0 https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip -O ~/data/workshop_data/occupancy_data.zip
! unzip ~/data/workshop_data/occupancy_data.zip -d ~/data/workshop_data/occupancy_data

In [ ]:
# Let's start by importing the relevant packages
# matplotlib for plots
import matplotlib as mpl
from matplotlib import pyplot as plt
# pandas to read in some data
import pandas as pd
# numpy to build our first perceptron
import numpy as np
# Train test split to do validate our findings from the perceptron training
from sklearn.model_selection import train_test_split
# MinMaxScaler to normalise the data before inputting them to the perceptron
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 9)
import os

home = os.path.expanduser("~")
data = home + '/data/workshop_data/occupancy_data/datatraining.txt'

## Occupancy Detection Dataset
For training the perceptron we will utilise the [occupancy detection dataset](https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.html?task=&area=&type=ts&view=table). It contatins experimental data for binary classification  if a person or not is in a room given temperature, humidity, light and CO$_2$.
	


In [ ]:
# Load the occupancy data so we have something to predict
df = pd.read_csv(data)
target = 'Occupancy'
features = [col for col in df.columns if target not in col and 'date' not in col]
df.head()

In [ ]:
print(df.min(), df.max())

We will normalize the data to be in a range from 0 to 1. This makes sure that all weights are in the same order of magnitude. Otherwise the perceptron would need to learn the range of the data first and then how to separate the data best.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df[features], df[target])
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Build the perceptron
To build and train a perceptron we have to perform three steps:
- Calculate the perceptron's output $\hat{y} = \left(\sum_i w_i X_i \geq 0\right)$ (this can be done in numpy using np.dot [docs](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html))
- Determine the update for the weights using the error and the learning rate: $\partial w_i = \alpha (y-\hat{y}) X_i$
- Calculate new weights as: $w_i \leftarrow w_i + \partial w_i$
- Repeat the above steps until there occur no more updates (we will iterate once over the dataset instead)

In [ ]:
# initializes weights
w = np.random.rand(len(features))
print("initial weights: {}".format(w))
# set a learning rare
alpha = 1e-2

In [ ]:
def calculate_perceptron_output(w, x):
    # Calculate the perceptrons output using 
    # np.dot(w, x) to calculate the sum and
    # thresholding the output:
    # solution: (np.dot(w, x) >= 0).astype(float)
    return (np.dot(w, x) >= 0).astype(float)

In [ ]:
for x, y in zip(x_train, y_train.values):
    y_hat = calculate_perceptron_output(w, x)
    error = y - unit_step(y_hat)
    # calculate delta_w
    delta_w = alpha * (y-y_hat) * x
    # update w
    w += delta_w

In [ ]:
results=[]
expected=[]
for x in x_train:
    results.append(calculate_perceptron_output(w, x))
results = np.array(results)
expected = np.array(y_train.values)
print("final weights: {}".format(w))

In [ ]:
plt.plot(expected-results, marker='.', ls='')

In [ ]:
print("accuracy: {}".format(np.mean(results == expected)))

## Let us change to PyTorch

In [ ]:
from torch import nn
import torch

Replace the calls to numpy with calls to torch and convert numpy arrays to torch tensors using torch_arr = torch.from_numpy(arr)

In [ ]:
def calculate_perceptron_output_torch(w, x):
    # Calculate the perceptrons output using 
    # np.dot(w, x) to calculate the sum and
    # thresholding the output:
    # solution: (np.dot(w, x) >= 0).astype(float)
    return torch.dot(w, x) >= 0

In [ ]:

w = torch.from_numpy(np.random.rand(len(features)))
alpha = torch.from_numpy(np.array(alpha))
print("initial weights: {}".format(w))
x_ttrain = torch.from_numpy(x_train)
y_ttrain = torch.from_numpy(y_train.values)
for x, y in zip(x_ttrain, y_ttrain):
    y_hat = calculate_perceptron_output_torch(w, x)
    error = y - y_hat
    w += alpha * error * x
print("final weights: {}".format(w))

In [ ]:
results=[]
expected=[]
for x, y in zip(x_ttrain, y_ttrain):
    result = calculate_perceptron_output_torch(w, x)
    expected.append(y)
    results.append(result)
results = torch.stack(results)
expected = torch.stack(expected)
print("weights: {}".format(w))
print("accuracy: {}".format((results == expected.byte()).float().mean()))

In [ ]:
ax = df[df.Occupancy==1].plot(x='CO2', y='Light', ls='', marker='o', ms=3, color='r', label='occupied')
df[df.Occupancy==0].plot(x='CO2', y='Light', ls='', marker='o', ms=3, color='b', ax=ax, label='empty')
ax.set_ylabel('Light')